In [1]:
!pip install transformers

In [2]:
from datasets import load_dataset

In [3]:
from datasets import load_dataset

#loading the twitter dataset
df = load_dataset("davidberg/sentiment-reviews")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

EcoPreprocessed.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/4084 [00:00<?, ? examples/s]

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

#loading the BERT tokenizer and BERT model for classification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def tokenize_function(examples):
    return tokenizer(examples['review'], padding="max_length", truncation=True)

# Applying tokenization to the dataset
tokenized_datasets = df.map(tokenize_function, batched=True)

Map:   0%|          | 0/4084 [00:00<?, ? examples/s]

In [6]:
print(tokenized_datasets['train'][0])

{'Unnamed: 0': 3870, 'review': 'able play youtube alexa', 'polarity': 0.5, 'division': 'positive', 'input_ids': [101, 2583, 2377, 7858, 24969, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [7]:
print(df.keys())

dict_keys(['train'])


In [8]:
from random import sample

# Check if 'test' split exists. If not, create it by splitting the 'train' dataset.
# The original dataset 'davidberg/sentiment-reviews' only has a 'train' split.
if "test" not in df:
    df = df["train"].train_test_split(test_size=0.33, seed=42) # Added seed for reproducibility

# Random samples for train and test from the (potentially new) splits
random_train_indices = sample(range(len(df["train"])), 1000)
random_test_indices = sample(range(len(df["test"])), 500)

small_train = df["train"].select(random_train_indices)
small_test = df["test"].select(random_test_indices)

# Performing tokenization on the random samples
tokenized_train = small_train.map(tokenize_function, batched=True)
tokenized_test = small_test.map(tokenize_function, batched=True)

# Add 'labels' column by mapping 'division' to 0 or 1
def map_labels_to_int(example):
    # 'positive' -> 1, 'negative' -> 0
    example['labels'] = 1 if example['division'] == 'positive' else 0
    return example

tokenized_train = tokenized_train.map(map_labels_to_int)
tokenized_test = tokenized_test.map(map_labels_to_int)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [9]:
#to calculate the metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import Trainer, TrainingArguments
import numpy as np

def compute_metrics(p):
    preds, labels = p
    preds = np.argmax(preds, axis=1)  # If the model is a classifier
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    accuracy = accuracy_score(labels, preds)
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [10]:
training_args = TrainingArguments(
    output_dir='./results',          # folder for results
    num_train_epochs=1,              # number of epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warm-up steps
    weight_decay=0.01,               # L2 regularization
    logging_dir='./logs',            # folder for logs
    logging_steps=10,
    report_to="none",                # Disable WandB and other integrations
)



In [11]:
trainer = Trainer(
    model=model,                         # the BERT model
    args=training_args,                  # hyperparameters
    train_dataset=tokenized_train,       # the training dataset
    eval_dataset=tokenized_test,         # the evaluation dataset
    compute_metrics=compute_metrics,     # Adding metric computation. We can run the algorithm even without this
)

In [12]:
trainer.train()

Step,Training Loss
10,1.312800
20,1.268200
30,1.148800
40,0.894700
50,0.724100
60,0.641500
70,0.582000
80,0.598900
90,0.478600
100,0.508600


TrainOutput(global_step=125, training_loss=0.7365702857971191, metrics={'train_runtime': 111.3565, 'train_samples_per_second': 8.98, 'train_steps_per_second': 1.123, 'total_flos': 263111055360000.0, 'train_loss': 0.7365702857971191, 'epoch': 1.0})

In [13]:
eval_results = trainer.evaluate()

# Printing the evaluation results
print(eval_results)

for key, value in eval_results.items():
    print(f"{key}: {value}")

{'eval_loss': 0.3805983364582062, 'eval_accuracy': 0.826, 'eval_precision': 0.8135964912280702, 'eval_recall': 0.9946380697050938, 'eval_f1': 0.8950542822677925, 'eval_runtime': 14.3566, 'eval_samples_per_second': 34.827, 'eval_steps_per_second': 2.229, 'epoch': 1.0}
eval_loss: 0.3805983364582062
eval_accuracy: 0.826
eval_precision: 0.8135964912280702
eval_recall: 0.9946380697050938
eval_f1: 0.8950542822677925
eval_runtime: 14.3566
eval_samples_per_second: 34.827
eval_steps_per_second: 2.229
epoch: 1.0


In [14]:
trainer.save_model("./model")
tokenizer.save_pretrained("./model")

('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/vocab.txt',
 './model/added_tokens.json')

In [16]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

# Load the model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("./model")
tokenizer = AutoTokenizer.from_pretrained("./model")

# Create a pipeline for sentiment analysis
sentiment_classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

# Example input for sentiment analysis
text = "Absolutely, mind blow device. It came in beautiful shape, excellent work and nice performance"
result = sentiment_classifier(text)

# Map label to human-readable sentiment
label_mapping = {'LABEL_0': 'negative', 'LABEL_1': 'positive'}
for r in result:
    r['label'] = label_mapping[r['label']]

print("Sentiment Analysis Result:", result)

Device set to use cuda:0


Sentiment Analysis Result: [{'label': 'positive', 'score': 0.922335684299469}]


In [18]:
#example 2
text = "It limit amazon apps music"
result = sentiment_classifier(text)

# Map label to human-readable sentiment
label_mapping = {'LABEL_0': 'negative', 'LABEL_1': 'positive'}
for r in result:
    r['label'] = label_mapping[r['label']]

print("Sentiment Analysis Result:", result)

Sentiment Analysis Result: [{'label': 'negative', 'score': 0.5341446399688721}]
